<a href="https://colab.research.google.com/github/jx-dohwan/DATATHON/blob/main/%ED%94%84%EB%9E%9C%EC%B0%A8%EC%9D%B4%EC%A6%88_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 커피, 분식집, 치킨 프랜차이즈 별 분포 히트맵
### 필요한 모듈 import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 소상공인 데이터 불러오기

In [4]:
data_dir = '/content/drive/MyDrive/DATATHON 공유 드라이브/DATATHON/소상공인시장진흥공단_상가(상권)정보_20220331/소상공인시장진흥공단_상가(상권)정보_서울_202203.csv'

data = pd.read_csv(data_dir, encoding='UTF-8')

In [23]:
data

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,23324279,제중건강원,NaN,D,소매,D10,건강/미용식품,D10A07,건강원,G47216,...,1156010600100410002034626,NaN,서울특별시 영등포구 영중로14길 11-17,150035,7250.0,NaN,1,NaN,126.907168,37.520613
1,24525909,민속악기사,NaN,D,소매,D04,취미/오락관련소매,D04A09,악기판매,G47593,...,1120012200101420011000227,NaN,서울특별시 성동구 용답5길 2,133849,4803.0,NaN,NaN,NaN,127.049018,37.566857
2,24715368,태평양진주,NaN,D,소매,D26,시계/귀금속소매,D26A01,시계/귀금속,G47830,...,1111015200100440000000001,금사랑투빌딩,서울특별시 종로구 서순라길 17-10,110390,3138.0,NaN,NaN,NaN,126.993530,37.571848
3,15554136,김선희꼼꼼국어교습소,NaN,R,학문/교육,R01,학원-보습교습입시,R01A01,학원-입시,P85501,...,1171011100101970003017195,NaN,서울특별시 송파구 위례성대로12길 31,138834,5640.0,NaN,2,NaN,127.121520,37.510967
4,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010100108540018009586,상랑의빌딩,서울특별시 서초구 동광로18길 82,137837,6572.0,NaN,1,NaN,126.991394,37.488375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330822,9032765,글로리아진스커피,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1165010300106860000000001,삼성프라자1,서울특별시 서초구 성촌1길 26,137901,6765.0,NaN,2,NaN,127.021354,37.465397
330823,9093595,김밥,NaN,Q,음식,Q04,분식,Q04A01,라면김밥분식,I56194,...,1171010800104090003000001,가든파이브툴,서울특별시 송파구 충민로 10,138200,5840.0,NaN,1,NaN,127.119371,37.478261
330824,9099241,더페이스샵아차산역점,아차산역점,D,소매,D16,화장품소매,D16A01,화장품판매점,G47813,...,1121510200102400029005869,아성빌라트,서울특별시 광진구 능동로32길 82-58,143849,4988.0,NaN,1,NaN,127.082117,37.552990
330825,9160309,CU자곡현대점,자곡현대점,D,소매,D03,종합소매점,D03A01,편의점,G47122,...,1168011200103950004000001,강남유탑유블레스,서울특별시 강남구 자곡로 180,135200,6373.0,NaN,1,NaN,127.103347,37.473707


### 업종 마다 분류하기
  - 대분류

In [7]:
category = data['상권업종대분류명'].value_counts()

category.keys()
for i in category.keys():
    print(i)

음식
소매
생활서비스
학문/교육
부동산
관광/여가/오락
스포츠
숙박


In [8]:
store = {}
for i in category.keys():
    store[i] = data.loc[data['상권업종대분류명']==i]
store['음식'].head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
4,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010100108540018009586,상랑의빌딩,서울특별시 서초구 동광로18길 82,137837,6572.0,NaN,1,NaN,126.991394,37.488375
5,17174119,쓰리에프,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1159010700101390073009536,NaN,서울특별시 동작구 동작대로27가길 12,156816,7008.0,NaN,NaN,NaN,126.980952,37.487105
6,17174096,채움,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1111012400100580000017956,두산위브파빌리온,서울특별시 종로구 삼봉로 81,110858,3150.0,NaN,NaN,NaN,126.981794,37.572387
7,17174040,다향,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010800114850004022127,NaN,서울특별시 서초구 효령로 230,137869,6709.0,NaN,1,NaN,127.009382,37.483436
8,25530299,고향생막걸리,NaN,Q,음식,Q09,유흥주점,Q09A04,민속주점,I56219,...,1168010700106150001009703,코끼리상가,서울특별시 강남구 압구정로 216,135894,6023.0,NaN,1,NaN,127.031012,37.528073


  - 프랜차이즈 별 점포 수

In [9]:
Franchise = store['음식']['상호명'].value_counts()

Franchise[1:21]

이디야커피       385
스타벅스        326
BBQ치킨       205
파리바게뜨       201
메가엠지씨커피     176
투썸플레이스      172
BHC치킨       169
컴포즈커피       164
나           162
김밥천국        149
커피빈         136
빽다방         124
뚜레쥬르        118
배스킨라빈스31    110
김가네         107
공차          105
까           104
본죽          103
교촌치킨         98
이삭토스트        96
Name: 상호명, dtype: int64

### 커피 프랜차이즈
  - 이디야
  - 스타벅스
  - 투썸 플레이스

In [16]:
# 이디야 데이터 분리
이디야 = store['음식'].loc[store['음식']['상호명'].values=='이디야커피']

# 히트맵 출력
m = folium.Map(location=[37.568016,126.987871], zoom_start=11.5)
히트맵_이디야 = HeatMap(zip(이디야['위도'], 이디야['경도']),
                  min_opacity=0.1,
                  max_val=5,
                  radius=10, blur=15,
                  max_zoom=5)
m.add_child(히트맵_이디야)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
1107,25435527,이디야커피,신길역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1156013200100590002019075,엔씨티오피스텔,서울특별시 영등포구 영등포로 353,150050,7318.0,NaN,1,NaN,126.918062,37.515118
1463,20015522,이디야커피,라이프점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1156011000100610003031503,라이프오피스텔,서울특별시 영등포구 63로 40,150731,7345.0,NaN,1,NaN,126.939198,37.519544
2426,20731751,이디야커피,시흥점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1154510300110110000004299,관악우방아파트,서울특별시 금천구 금하로 750,153784,8645.0,1,NaN,NaN,126.913646,37.449066
4244,20742476,이디야커피,개봉중앙점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1153010700103580020027286,NaN,서울특별시 구로구 개봉로3길 26,152090,8352.0,NaN,1,NaN,126.854872,37.486436
5404,23412756,이디야커피,해군회관점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1159010800103850024004968,NaN,서울특별시 동작구 여의대방로 194,156810,6945.0,NaN,1,NaN,126.922245,37.504253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311278,18199400,이디야커피,방배백석점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1165010100110230001011526,NaN,서울특별시 서초구 방배로 42,137851,6705.0,NaN,1,NaN,126.999766,37.478714
311554,17479944,이디야커피,강북경찰서점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1130510200104450025017631,NaN,서울특별시 강북구 오패산로 403,142867,1065.0,NaN,NaN,NaN,127.026984,37.636868
316487,16406711,이디야커피,만리동고개점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1114017400100370002001169,서울역센트럴자이,서울특별시 중구 만리재로 175,100858,4500.0,NaN,NaN,NaN,126.964594,37.554739
321643,17065461,이디야커피,문정문현점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1171010800100750009003428,NaN,서울특별시 송파구 새말로8길 30,138868,5817.0,NaN,NaN,NaN,127.126630,37.480126


In [18]:
# 스타벅스 데이터 분리
스타벅스 = store['음식'].loc[store['음식']['상호명'].values=='스타벅스']

# 히트맵 출력
히트맵_스타벅스 = HeatMap(zip(스타벅스['위도'], 스타벅스['경도']),
                  min_opacity=0.1,
                  max_val=5,
                  radius=10, blur=15,
                  max_zoom=5)
m.add_child(히트맵_스타벅스)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
1512,20585779,스타벅스,동숭로아트점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1111016800100300000009771,NaN,서울특별시 종로구 동숭길 110,110809,3085.0,NaN,3,NaN,127.003887,37.582964
2533,20666449,스타벅스,현대목동점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1147010200109160000021253,하이페리온,서울특별시 양천구 목동동로 257,158724,7998.0,NaN,2,NaN,126.874682,37.527147
3553,22934175,스타벅스,가로수길점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1168010700105290010010670,NaN,서울특별시 강남구 논현로175길 94,135888,6028.0,NaN,2,NaN,127.021629,37.523184
3631,23320126,스타벅스,당산점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1156011600103380011027353,한경빌딩,서울특별시 영등포구 양평로 64,150810,7222.0,NaN,1,NaN,126.899952,37.535134
4617,23456329,스타벅스,문래역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1156012100100550005037427,로데오샵쇼핑몰,서울특별시 영등포구 당산로 34,150835,7297.0,NaN,1,NaN,126.896479,37.517450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326379,17447086,스타벅스,대치한티점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1168010600109220001013988,SH타워,서울특별시 강남구 선릉로 324,135843,6205.0,NaN,2,NaN,127.052091,37.498518
326895,17466170,스타벅스,영등포시장교차로점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1156010700100010014033467,NaN,서울특별시 영등포구 영중로 45,150030,7251.0,NaN,1,NaN,126.904444,37.520452
327015,17471091,스타벅스,선유도역1번출구점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1156012900100640001020479,NaN,서울특별시 영등포구 양평로 128,150105,7205.0,NaN,2,NaN,126.893648,37.538282
327419,17484396,스타벅스,서울역사점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1117010700100430205024398,서울역(철도역),서울특별시 용산구 한강대로 405,140761,4320.0,NaN,2,NaN,126.971712,37.554828


In [ ]:
# 투썸 플레이스 데이터 분리
투썸플레이스 = store['음식'].loc[store['음식']['상호명'].values=='투썸플레이스']

# 히트맵 출력
히트맵_투썸플레이스 = HeatMap(zip(투썸플레이스['위도'], 투썸플레이스['경도']),
                  min_opacity=0.1,
                  max_val=5,
                  radius=10, blur=15,
                  max_zoom=5)
m.add_child(히트맵_투썸플레이스)

### 치킨 프랜차이즈
  - BBQ
  - BHC
  - 교촌 치킨

In [20]:
# BBQ 데이터 분리
bbq = store['음식'].loc[store['음식']['상호명'].values=='bbq']

# 히트맵 출력
히트맵_bbq = HeatMap(zip(bbq['위도'], bbq['경도']),
                  min_opacity=0.1,
                  max_val=5,
                  radius=10, blur=15,
                  max_zoom=5)
m.add_child(히트맵_bbq)

In [25]:
# BHC 데이터 분리
bhc = store['음식'].loc[store['음식']['상호명'].values=='BHC치킨']

# 히트맵 출력
히트맵_bhc = HeatMap(zip(bhc['위도'], bhc['경도']),
                  min_opacity=0.1,
                  max_val=5,
                  radius=10, blur=15,
                  max_zoom=5)
m.add_child(히트맵_bhc)

In [ ]:
# 교촌 치킨 데이터 분리
교촌치킨 = store['음식'].loc[store['음식']['상호명'].values=='교촌치킨']

# 히트맵 출력
히트맵_교촌치킨 = HeatMap(zip(교촌치킨['위도'], 교촌치킨['경도']),
                  min_opacity=0.1,
                  max_val=5,
                  radius=10, blur=15,
                  max_zoom=5)
m.add_child(히트맵_교촌치킨)

### 분식 프랜차이즈
  - 김밥천국

In [ ]:
# 김밥천국 데이터 분리
김밥천국 = store['음식'].loc[store['음식']['상호명'].values=='김밥천국']

# 히트맵 출력
히트맵_김밥천국 = HeatMap(zip(김밥천국['위도'], 김밥천국['경도']),
                  min_opacity=0.1,
                  max_val=5,
                  radius=10, blur=15,
                  max_zoom=5)
m.add_child(히트맵_김밥천국)

### 번외
  - 베스킨라빈스

In [22]:
# 베스킨라빈스 데이터 분리
배스킨라빈스 = store['음식'].loc[store['음식']['상호명'].values=='배스킨라빈스']

# 히트맵 출력
히트맵_배스킨라빈스 = HeatMap(zip(배스킨라빈스['위도'], 배스킨라빈스['경도']),
                  min_opacity=0.1,
                  max_val=5,
                  radius=10, blur=15,
                  max_zoom=5)
m.add_child(히트맵_배스킨라빈스)